In [ ]:
from types import SimpleNamespace
import numpy as np
import holoviews as hv
import pandas as pd
hv.notebook_extension()
%opts Scatter (color='b') 
d = SimpleNamespace(phi=hv.Dimension(r'$\phi$', unit='rad'),
                    phi_c=hv.Dimension(r'$\phi_c$', unit='rad'),
                    I=hv.Dimension('$I$', unit=r'meV / $\Phi_0$'),
                    I_c=hv.Dimension('$I_c$', unit=r'meV / $\Phi_0$'),
                    B=hv.Dimension('$B$', unit='T'),
                    T=hv.Dimension('$T$', unit='K'), 
                    L=hv.Dimension('$L$', unit='nm'), 
                    g=hv.Dimension('$g$-factor'), 
                    alpha=hv.Dimension(r'$\alpha$', unit='mev nm'), 
                    mu=hv.Dimension(r'$\mu$', unit='meV'), 
                    a=hv.Dimension('$a$', unit='nm'))

pd.set_option('display.max_columns', None)

In [ ]:
df = pd.read_hdf('all_data.hdf')
inds = ['T', 'L', 'with_leads', 'orbital', 'g', 'alpha', 'mu', 'a']
kdims = [d.T, d.L, 'infinite leads', 'orbital', d.g, d.alpha, d.mu, d.a]
gb = df.groupby(inds)
groups = dict(list(gb))

In [ ]:
def current_curve(T, L, leads, orbital, g, alpha, mu, a):
    gr = groups[(T, L, leads, orbital, g, alpha, mu, a)]
    return hv.Path((gr['B_x'], gr['current_c']), kdims=[d.B, d.I_c])[:, 0:df['current_c'].max()]

def phase_curve(T, L, leads, orbital, g, alpha, mu, a):
    gr = groups[(T, L, leads, orbital, g, alpha, mu, a)]
    return hv.Path((gr['B_x'], gr['phase_c']), kdims=[d.B, d.phi_c])[:, np.pi:np.pi]

kdims = [hv.Dimension('T', values=df['T'].unique().tolist()),
         hv.Dimension('L', values=df['L'].unique().tolist()),
         hv.Dimension('leads', values=[True, False]),
         hv.Dimension('orbital', values=[True, False]),
         hv.Dimension('g', values=df['g'].unique().tolist()),
         hv.Dimension('alpha', values=df['alpha'].unique().tolist()),
         hv.Dimension('mu', values=df['mu'].unique().tolist()),
         hv.Dimension('a', values=[8])]

hv.DynamicMap(current_curve, kdims=kdims) + hv.DynamicMap(phase_curve, kdims=kdims)